
# Agno Teams — Concepts, Flows & Operations (with Mermaid)

*Updated:* 2025-10-25 20:58 UTC  
This notebook explains **Agno Teams** from fundamentals to advanced operations.  
Diagrams are written in **Mermaid** so you can render them in environments that support it.

**Scope covered:** Teams ▸ Building ▸ Running ▸ Delegation ▸ Debugging ▸ Conversation History ▸ Team Sessions ▸ Context Engineering ▸ Input & Output ▸ Dependencies ▸ Shared State ▸ Storage ▸ Memory ▸ Knowledge ▸ Pre/Post‑hooks ▸ Guardrails ▸ Metrics ▸ Cancelling a Run ▸ Custom Loggers



## 1. What is a Team?

A **Team** is a coordinator (team leader) plus a set of **members** (Agents or sub‑Teams).  
The leader routes work, gathers the results, and returns a single, coherent answer (unless configured to return member replies directly).

```mermaid
flowchart LR
    U[User Input] --> L[Team Leader]
    subgraph T[Team]
      L -->|delegates| A1[Agent: News]
      L -->|delegates| A2[Agent: Weather]
      L -->|delegates| S1[Sub‑Team]
      subgraph S1[Sub‑Team: Research]
        R1[Agent: Web] --> R2[Agent: Summary]
      end
    end
    A1 --> L
    A2 --> L
    S1 --> L
    L --> O[Final Response]
```



### Core Capabilities (Quick Map)

| Capability | What it controls | Typical flag(s) / knobs |
|---|---|---|
| Model | LLM used by leader (and optionally members) | `model` on Team/Agent |
| Instructions | How the leader should behave | `instructions`, `description`, `expected_output` |
| Tools | Functions/APIs available | `tools` (team or member) |
| Delegation | Whether/how to delegate | `respond_directly`, `delegate_task_to_all_members`, `determine_input_for_members` |
| History | Past turns in context | `db`, `add_history_to_context`, `num_history_runs` |
| Memory | Long‑lived user facts | `enable_user_memories` |
| Knowledge | Retrieval‑augmented grounding | `add_knowledge_to_context`, KB tooling |
| State | Shared mutable data per session | `session_state`, `enable_agentic_state` |
| Debugging | Rich traces & events | `debug_mode`, `debug_level`, streaming events |



## 2. Building a Team (minimal)

Keep it simple: leader model, a couple of focused members, and a short instruction.

```python
# (illustrative) minimal setup — adapt IDs to your environment
from agno.team import Team
from agno.agent import Agent
from agno.models.openai import OpenAIChat

news = Agent(name="News",  role="Fetch and summarize current news")
wx   = Agent(name="Weather", role="Provide concise forecasts")

team = Team(
    name="News+Weather",
    members=[news, wx],
    model=OpenAIChat(id="gpt-4o"),
    instructions=["Answer the user's request by delegating appropriately. Keep it brief."]
)
```



## 3. Running a Team

Use `run()` for a single response or stream events for live updates.

```python
# Single response
resp = team.run("What's happening in Tokyo and how's the weather?")
print(resp.content)

# Streaming (yields TeamRunOutputEvent chunks)
for ev in team.run("Same question, stream it", stream=True, stream_events=True):
    if getattr(ev, "content", None):
        print(ev.content, end="")
```



## 4. How Delegation Works

The leader may reply directly or use the internal **delegate_task_to_members** tool to route tasks.  
Members can run concurrently when using async execution and multi‑member delegation.

```mermaid
sequenceDiagram
    participant U as User
    participant L as Team Leader
    participant N as News Agent
    participant W as Weather Agent
    U->>L: Ask: "Latest news and 3‑day forecast for Tokyo"
    L->>N: Task: "Summarize latest Tokyo news"
    L->>W: Task: "Get concise 3‑day forecast"
    par Concurrent
        N-->>L: News summary
        W-->>L: Forecast
    end
    L-->>U: Synthesized answer
```



### Useful Delegation Knobs

- **`respond_directly=True`** → leader returns member outputs as‑is (good for router patterns).
- **`determine_input_for_members=False`** → user input is passed verbatim to selected members.
- **`delegate_task_to_all_members=True`** → broadcast a task to every member at once (often with `arun`).

```mermaid
flowchart TB
    A[User Input] --> L[Leader]
    L -->|respond_directly| M1[Member A]
    L -->|respond_directly| M2[Member B]
    M1 --> O1[Direct Out]
    M2 --> O2[Direct Out]
```



## 5. Debugging Teams

Enable **debug mode** for verbose assembly of prompts, tool calls and metrics. Use streaming to observe events in real time.

```python
team = Team(
    name="Debugger",
    members=[news, wx],
    model=OpenAIChat(id="gpt-4o"),
    debug_mode=True,      # set debug_level=2 for even more detail
    show_members_responses=True
)
```



## 6. Conversation History

With a database attached, the team can bring **recent turns** into context.

```python
from agno.db.sqlite import SqliteDb
team = Team(
    members=[news, wx],
    model=OpenAIChat(id="gpt-4o"),
    db=SqliteDb(db_file="tmp/team.db"),
    add_history_to_context=True,
    num_history_runs=3
)
```
**Tip:** More history ⇒ bigger prompts. Start small and increase as needed.



## 7. Team Sessions

Sessions group multiple runs for continuity. Use `user_id` and `session_id` to separate users and threads.

```python
team.run("Bonjour !", user_id="u1", session_id="s1")
team.run("Tell me a joke.", user_id="u1", session_id="s1")
team.run("Start fresh.", user_id="u1", session_id="s2")
```
Optionally enable **session summaries** to condense long histories.



## 8. Context Engineering (system message)

The system message is composed from description, instructions, member info, and optional add‑ons (time, location, session summary, memories, etc.).

```mermaid
flowchart LR
    D[Description] --> S[(System Message)]
    I[Instructions] --> S
    M[Member Info & Tools] --> S
    X[Add‑ons: time/location/memories/summary/state] --> S
    S --> Model
```
**Common parameters:** `description`, `instructions`, `expected_output`, `markdown`, `add_datetime_to_context`, `add_member_tools_to_context`, `add_session_summary_to_context`, `add_memories_to_context`.



## 9. Input & Output (structured when needed)

Teams accept plain strings or **structured input** via Pydantic; outputs can also be validated.

```python
from pydantic import BaseModel

class StockReport(BaseModel):
    symbol: str
    company_name: str
    analysis: str

team = Team(
    name="Stocks",
    members=[],
    model=OpenAIChat(id="gpt-4o"),
    output_schema=StockReport,   # enforce structured output
)
result = team.run("Analyze NVDA in ~4 lines").content
```



## 10. Dependencies (context injection)

Provide callable dependencies to inject dynamic values (profile, time, retrieved docs) before each run.

```python
def get_profile():
    return {"name": "Ava", "interests": ["AI", "finance"]}

team = Team(
    members=[news, wx],
    model=OpenAIChat(id="gpt-4o"),
    dependencies={"user_profile": get_profile},
    instructions=[
        "Personalize answers using {user_profile}. Keep responses concise."
    ]
)
```



## 11. Shared State (per‑session)

A shared dictionary lives with the session. Tools receive it automatically so members can coordinate.

```mermaid
flowchart LR
    SS[(session_state)] --- A1[Member Tool]
    SS --- A2[Team Tool]
    U[User] --> Leader
    Leader -->|updates| SS
    A1 -->|reads/writes| SS
    A2 -->|reads/writes| SS
```
```python
def add_item(session_state, item: str):
    if item not in session_state.setdefault("shopping_list", []):
        session_state["shopping_list"].append(item)
        return f"Added {item}"
    return f"{item} already present"
```



## 12. Storage (sessions & state persistence)

Attach a database so sessions and state survive across requests.

```python
from agno.db.postgres import PostgresDb
db = PostgresDb(db_url="postgresql+psycopg://ai:ai@localhost:5532/ai")
team = Team(members=[news, wx], db=db)
```

**Session table (conceptual):**

| Field | Type | Notes |
|---|---|---|
| `session_id` | `str` | Unique key |
| `session_type` | `str` | Team/Agent/Workflow |
| `agent_id` | `str` | If created by an Agent |
| `team_id` | `str` | If created by a Team |
| `workflow_id` | `str` | If created by a Workflow |
| `user_id` | `str` | End‑user identifier |
| `session_data` | `dict` | Arbitrary session payload |
| `agent_data` / `team_data` / `workflow_data` | `dict` | Component‑specific |
| `metadata` | `dict` | Custom tags |
| `runs` | `list` | Run history |
| `summary` | `dict` | Optional session summary |
| `created_at` / `updated_at` | `int` | Timestamps |



## 13. Memory (long‑lived user facts)

Enable user memories so the team can recall details across sessions.

```python
from agno.db.sqlite import SqliteDb
team_with_memory = Team(
    name="WithMemory",
    members=[news, wx],
    db=SqliteDb(db_file="agno.db"),
    enable_user_memories=True,
)
```



## 14. Knowledge (RAG)

Give the leader access to a knowledge base or search tools and include retrieved snippets in context.
Key toggles include `add_knowledge_to_context=True` and toolkits (e.g., web search, DB readers).
Use **agentic knowledge filters** to let the leader choose metadata filters when searching.



## 15. Pre‑hooks & Post‑hooks (and Guardrails)

**Pre‑hooks** run before the model — validate inputs, strip secrets, reshape data.  
**Post‑hooks** run after the model — validate/transform output or enforce policies.  
**Guardrails** are a common pre‑hook pattern to block disallowed content.

```python
# Tiny pre‑hook example (conceptual)
from agno.exceptions import InputCheckError, CheckTrigger
def block_empty(run_input, session, **_):
    if not (run_input and getattr(run_input, "input_content", "").strip()):
        raise InputCheckError("Empty input not allowed", check_trigger=CheckTrigger.INPUT_NOT_ALLOWED)
```



## 16. Metrics

Every run exposes token/time metrics at message, member, team and session levels.

```python
out = team.run("Quick test")
print("Team totals:", out.metrics)
if out.messages:
    for m in out.messages:
        if getattr(m, "metrics", None):
            print("Per‑message:", m.metrics)
```
```mermaid
flowchart LR
    Msg[Message Metrics] --> Agg[Team Aggregates]
    Mem[Member Metrics] --> Agg
    Agg --> Sess[Session Aggregates]
```



## 17. Cancelling a Run

Long operations can be cancelled by `team.cancel_run(run_id)` during streaming from another thread/process.
Keep the `run_id` you receive as soon as streaming begins.



## 18. Custom Loggers

Swap Agno's default logger for your own (JSON, SIEM, etc.).

```python
from agno.utils.log import configure_agno_logging
import logging

custom = logging.getLogger("my_team_logger")
custom.setLevel(logging.INFO)
configure_agno_logging(custom_team_logger=custom)
```



## 19. Summary

- Treat the Team Leader as a **router + synthesizer**.  
- Keep builds **minimal** first; add state/history/memory only when needed.  
- Prefer **structured I/O** for reliability in production.  
- **Mermaid diagrams** here outline flows you can adapt to your architecture.  
- Use **hooks, guardrails, metrics and logging** to make systems production‑ready.


In [ ]:

# Optional: lightweight smoke‑test to ensure Python imports resolve in your env.
# Comment out if you haven't installed agno and providers yet.
try:
    import agno  # noqa: F401
    print("Agno import OK.")
except Exception as e:
    print("Agno not installed in this environment (that's fine for reading).")
